In [7]:
import pickle
import matplotlib.pyplot as plt
from collections import Counter
import math

In [8]:

# Load the data
with open('./Data/test_pride_prejudice.pkl', 'rb') as f:
    test_pride_prejudice = pickle.load(f)

with open('./Data/train_pride_prejudice.pkl', 'rb') as f:
    train_pride_prejudice = pickle.load(f)

with open('./Data/vocab_pride_prejudice.pkl', 'rb') as f:
    vocab_pride_prejudice = pickle.load(f)

with open('./Data/vocab_ulysses.pkl', 'rb') as f:
    vocab_ulysses = pickle.load(f)

with open('./Data/test_ulysses.pkl', 'rb') as f:
    test_ulysses = pickle.load(f)

with open('./Data/train_ulysses.pkl', 'rb') as f:
    train_ulysses = pickle.load(f)


In [9]:
# DECLARING SOME GLOBAL VARIABLES WHICH WILL BE USED IN THE CODE

# dictionary with key having the N of N gram
all_n_grams = {1: [], 2: [], 3: [], 4: [], 5: []}
# the number of unique words before which the sequence occurs (the numerator in (1- lambda) expression)
unique_n_grams = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}}
# dictionary with key having the N of N gram,  which itself is a dictionary of the count of that N-gram (the other denominator in the (1-lambda) expression)
n_gram_counts = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}}
probabilities = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}}
all_tokens_vocab = []
all_tokens_in_corpus = []
witten_bell_probabilities = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}}

# therefore
# (1-lambda) = unique_n_grams / (n_gram_counts + unique_n_grams)


In [10]:
def convert_to_sentence_tokens(corpus, n):
    """
    This function converts the corpus into a list of sentences, where each sentence is a list of tokens
    :param corpus: the corpus
    :return: list of sentences, where each sentence is a list of tokens
    """
    sentences_tokens = []
    intial_tokens = "<START> "*(n-1)
    intial_tokens = intial_tokens.split()
    sentences_tokens = sentences_tokens + \
        [intial_tokens+s.split()+["<END>"] for s in corpus]
        
    return sentences_tokens

In [11]:
def find_probablity(n):
    """
    This function finds the probability of each n-gram in the corpus
    """
    if (n == 1):
        # for unigrams the probability will be the count of the word divided by the total number of words in the corpus
        for n_gram in n_gram_counts[n]:
            probability = n_gram_counts[n][n_gram]/len(all_tokens_in_corpus)
            probabilities[n].update({n_gram: probability})
            # print("Added probability for n_gram: ", n_gram, " with probability: ", probability)
    else:
        # for all other n-gram the probability will be the count of the n-gram divided by the count of the n-1 gram
        for n_gram in n_gram_counts[n]:
            history = n_gram[:(n-1)]
            probability = n_gram_counts[n][n_gram]/n_gram_counts[n-1][history]
            probabilities[n].update({n_gram: probability})

In [12]:
def find_ngrams(n, input_list):  # go inside the list of sentences
    """
    This function finds all the n-grams in the corpus
    """
    for i in range(len(input_list)):  # for each sentence find all the n-grams
        for j in range(len(input_list[i])-n+1):
            ngram = tuple(input_list[i][j:j+n])
            all_n_grams[n].append(ngram)


In [13]:

def common_data(n, train_data, train_data_vocab):

    sentences_tokens = convert_to_sentence_tokens(train_data, n)

    all_tokens_vocab.extend(train_data_vocab)
    all_tokens_in_corpus.extend(
        [token for s in sentences_tokens for token in s])

    for i in range(1, 5):
        find_ngrams(i, sentences_tokens)
        n_gram_counts[i] = dict(Counter(all_n_grams[i]))
        unique_n_grams[i-1] = dict(Counter([i[:-1]
                                   for i in n_gram_counts[i].keys()]))
        find_probablity(i)

In [14]:
def empty_dict():
    for i in range(1, 6):
        all_n_grams[i] = []
        n_gram_counts[i] = {}
        unique_n_grams[i] = {}
        probabilities[i] = {}
        witten_bell_probabilities[i] = {}

In [15]:
# common_data(4, train_pride_prejudice, vocab_pride_prejudice)

In [16]:
# all_n_grams[4]
# n_gram_counts[3]
# unique_n_grams[2]
# probabilities[3]

In [17]:
# fig, ax= plt.subplots()
# for i in range(1, 6):
#     ax.scatter(i, len(n_gram_counts[i]))
#     ax.stem(i, len(unique_n_grams[i-1]))
# plt.show()

---

## Witten Bell Smoothing

It is the nth-order smoothed model is defiined recursively as a linear interpolation between the nth-order maximum likelihood model and the (n - 1)th-order smoothed model as in equation
<br><br>
<img src="./images/witten1.png" alt="Witten Bell Formula">

To compute the parameters lambda for Witten-Bell smooothing, we will need to use the number of unique words that follow the history w (i-1),(i-n+1) . We will write this value as N1+ (w (i-1), (i-n+1)), formally defined as
<br><br>
<img src="./images/witten2.png" alt="Witten Bell Formula">


The notation N1+ is meant to evoke the number of words that have one or more counts, and the black dot is meant to evoke a free  variable that is summed over. We can then assign the parameters lambda (i-1), (i-n+1) for Witten-Bell smoothing such that
<br><br>
<img src="./images/witten3.png" alt="Witten Bell Formula">

Substituting, we can rewrite the equaltion as
<br><br>
<img src="./images/witten4.png" alt="Witten Bell Formula">


---

In [18]:
empty_dict()
common_data(4, train_pride_prejudice, vocab_pride_prejudice)

In [19]:

def find_lambda(n, history):
    unique_n_grams[n-1].setdefault(history, 0)
    n_gram_counts[n-1].setdefault(history, 0)
    # if the history is not present in the corpus then we will assign zero probability to the current n-gram and shift all the mass to the next (n-1)-gram
    if ((unique_n_grams[n-1][history]+n_gram_counts[n-1][history]) == 0):
        return 0
    
    lamda = 1-(unique_n_grams[n-1][history]/(n_gram_counts[n-1]
               [history]+unique_n_grams[n-1][history]))
    return lamda


In [20]:
def witten_bell_for_one(n,  tuple_n):
    history = tuple_n[:(n-1)]
    if (n == 1):
        n_gram_counts[n].setdefault(tuple_n, 1)
        witten_probability= n_gram_counts[n][tuple_n]/len(unique_n_grams[n].keys())
        # witten_probability = n_gram_counts[n][tuple_n]/len(all_tokens_in_corpus) + (1 / (len(all_tokens_in_corpus)+len(all_tokens_vocab)))
        witten_bell_probabilities[n].update({tuple_n: witten_probability})
        return witten_probability

    lamda = find_lambda(n, history)
    probabilities[n].setdefault(tuple_n, (1/len(probabilities[n].keys())))
    witten_probability = probabilities[n][tuple_n] * \
        lamda + (1-lamda)*witten_bell_for_one(n-1, tuple_n[1:])
    witten_bell_probabilities[n].update({tuple_n: witten_probability})

    return witten_probability


In [21]:
def witten_bell(n):
    for tuplee in n_gram_counts[n].keys():
        witten_bell_for_one(n,tuplee)

In [22]:
witten_bell(4)
# probabilities[4]
witten_bell_probabilities[4]

{('<START>', '<START>', '<START>', 'pride'): 0.0006575282474925164,
 ('<START>', '<START>', 'pride', '&'): 0.41670046154046514,
 ('<START>', 'pride', '&', 'prejudice'): 0.9444739889857949,
 ('pride', '&', 'prejudice', '<END>'): 0.9751453314454371,
 ('<START>', '<START>', '<START>', 'it'): 0.03352607373542354,
 ('<START>', '<START>', 'it', 'is'): 0.30942756616136785,
 ('<START>', 'it', 'is', 'a'): 0.15004926935921323,
 ('it', 'is', 'a', 'truth'): 0.035033102894129454,
 ('is', 'a', 'truth', 'universally'): 0.7576120167855969,
 ('a', 'truth', 'universally', 'acknowledged'): 0.7919857477132525,
 ('truth', 'universally', 'acknowledged', ','): 0.9361531098129694,
 ('universally', 'acknowledged', ',', 'that'): 0.6036332661429482,
 ('acknowledged', ',', 'that', 'a'): 0.26068606440902653,
 (',', 'that', 'a', 'single'): 0.12348958929611215,
 ('that', 'a', 'single', 'man'): 0.6144499512399959,
 ('a', 'single', 'man', 'in'): 0.3992974405896449,
 ('single', 'man', 'in', 'possession'): 0.54577153288

In [23]:
log_probabilities_witten = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}}
for i in range(1, 5):
    for key in witten_bell_probabilities[i].keys():
        log_probabilities_witten[i].update(
            {key: math.log(witten_bell_probabilities[i][key])})

In [24]:
log_probabilities_witten[4]

{('<START>', '<START>', '<START>', 'pride'): -7.327022832869389,
 ('<START>', '<START>', 'pride', '&'): -0.875387632945835,
 ('<START>', 'pride', '&', 'prejudice'): -0.05712713187368684,
 ('pride', '&', 'prejudice', '<END>'): -0.025168761199557722,
 ('<START>', '<START>', '<START>', 'it'): -3.395431822425009,
 ('<START>', '<START>', 'it', 'is'): -1.1730312492677681,
 ('<START>', 'it', 'is', 'a'): -1.8967915764230894,
 ('it', 'is', 'a', 'truth'): -3.35146186750174,
 ('is', 'a', 'truth', 'universally'): -0.2775838755866481,
 ('a', 'truth', 'universally', 'acknowledged'): -0.23321188264117867,
 ('truth', 'universally', 'acknowledged', ','): -0.06597623702209784,
 ('universally', 'acknowledged', ',', 'that'): -0.5047884407128879,
 ('acknowledged', ',', 'that', 'a'): -1.3444384139822547,
 (',', 'that', 'a', 'single'): -2.091598423664786,
 ('that', 'a', 'single', 'man'): -0.48702779960706183,
 ('a', 'single', 'man', 'in'): -0.9180486746764418,
 ('single', 'man', 'in', 'possession'): -0.60555

In [25]:
def Get_perplexity(n, sentences_tokens, log_probabilities):
    all_perplexity = []
    for sentence in sentences_tokens:
        # print("------SENTENCE------\n", sentence)
        single_perplex = 0
        for i in range(len(sentence) - n + 1):
            if tuple(sentence[i:i+n]) in log_probabilities[4].keys():
                # print("Found earlier : ", tuple(sentence[i:i+n]))
                single_perplex += log_probabilities[4][tuple(sentence[i:i+n])]
                # print("Found probability : ",
                    #   witten_bell_probabilities[4][tuple(sentence[i:i+n])])
            else:
                # print("Not found earlier : ", tuple(sentence[i:i+n]))
                log_probabilities[4].setdefault(
                    tuple(sentence[i:i+n]), (1/len(log_probabilities[4].keys())))
                log_probabilities[4][tuple(sentence[i:i+n])] = math.log(witten_bell_for_one(n, tuple(sentence[i:i+n])))
                single_perplex += log_probabilities[4][tuple(sentence[i:i+n])]
            # print(sentence[i:i+n]," : ", math.exp(single_perplex*-1))

        single_perplex /= (len(sentence) - n + 1)
        single_perplex *= -1
        single_perplex = math.exp(single_perplex)
        all_perplexity.append(single_perplex)
    return all_perplexity


In [26]:
test_sentence_tokens=convert_to_sentence_tokens(test_pride_prejudice,4)
pp = Get_perplexity(4, test_sentence_tokens, log_probabilities_witten)
print('Perplexity : ', sum(pp)/len(pp))

Perplexity :  149.04806202021925


# KNESER-NEY

<img src="./images/kneser1.png" alt="Kneser-Ney ">

In [27]:
empty_dict()
common_data(4, train_pride_prejudice, vocab_pride_prejudice)

In [28]:
discount = 0.9
kneserney_probabilities = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}}

In [29]:

def kneserney_for_one(n, tuple_n):
    history = tuple_n[:(n-1)]
    if (n == 1):
        n_gram_counts[n].setdefault(tuple_n, 1)
        kneserney_probability = n_gram_counts[n][tuple_n] / len(unique_n_grams[n].keys())
        kneserney_probabilities[n].update({tuple_n: kneserney_probability})
        return kneserney_probability

    if (tuple_n not in n_gram_counts[n].keys()):
        n_gram_counts[n].update({tuple_n: 1})
        
    if (n_gram_counts[n-1][history] == 0 or history not in n_gram_counts[n-1].keys()):
        n_gram_counts[n-1].update({history: 1})
        
    unique_n_grams[n-1].setdefault(history, 1)
        
    kneserney_probability = max(0, (n_gram_counts[n][tuple_n]-discount))/n_gram_counts[n-1][history] + (
        (discount*unique_n_grams[n-1][history])/n_gram_counts[n-1][history])*kneserney_for_one(n-1, tuple_n[1:])
    kneserney_probabilities[n].update({tuple_n: kneserney_probability})

    return kneserney_probability


In [30]:
def kneserney(n):
    for tuplee in n_gram_counts[n].keys():
        kneserney_for_one(n,tuplee)
        
kneserney(4)

In [31]:
log_probabilities_kneserney = {1: {}, 2: {}, 3: {}, 4: {}, 5: {}}

for i in range(1, 5):
    for key in kneserney_probabilities[i].keys():
        log_probabilities_kneserney[i].update(
            {key: math.log(kneserney_probabilities[i][key])})
        

In [32]:
def Get_perplexity(n, sentences_tokens, log_probabilities):
    all_perplexity = []
    for sentence in sentences_tokens:
        # print("------SENTENCE------\n", sentence)
        single_perplex = 0
        for i in range(len(sentence) - n + 1):
            if tuple(sentence[i:i+n]) in log_probabilities[n].keys():
                single_perplex += log_probabilities[n][tuple(sentence[i:i+n])]
            else:

                log_probabilities[n][tuple(sentence[i:i+n])] = math.log(kneserney_for_one(n, tuple(sentence[i:i+n])))
                single_perplex += log_probabilities[4][tuple(sentence[i:i+n])]

        single_perplex /= (len(sentence) - n + 1)
        single_perplex *= -1
        single_perplex = math.exp(single_perplex)
        all_perplexity.append(single_perplex)
    return all_perplexity

In [33]:
test_sentence_tokens=convert_to_sentence_tokens(test_pride_prejudice,4)
pp = Get_perplexity(4, test_sentence_tokens, log_probabilities_kneserney)
print('Perplexity : ', sum(pp)/len(pp))

Perplexity :  8.76409478342123
